## Imports


In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from scipy.stats import poisson
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from datetime import datetime
from datetime import date
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import sasoptpy as so
import os
import warnings
from IPython.display import clear_output
from IPython.display import HTML
from pathlib import Path


pd.set_option('display.max_columns', 300)

## Fixture Generating Funtions
Generate ticker for visualization and optimization, based on the live fixture info from the Premier League API and any custom fixture timings and probabilities set by the user

In [2]:
## Generate matchday ticker dataframe, team_fixtures
def generate_ticker(gw_range=None, exclude_teams=None, custom_fixtures=None, extra_fixtures=None, generate_all_dataframes=False):

    # Infer fixture difficulties
    team_priors = pd.read_csv(f'../data/team_priors.csv')
    team_priors['h_off'] = round(team_priors['bl_g_for'] * team_priors['home_adv_g'],2)
    team_priors['h_def'] = round(team_priors['bl_g_against'] / team_priors['home_adv_g'],2)
    team_priors['h_gd'] = team_priors['h_off'] - team_priors['h_def']
    team_priors['a_off'] = round(team_priors['bl_g_for'] / team_priors['home_adv_g'],2)
    team_priors['a_def'] = round(team_priors['bl_g_against'] * team_priors['home_adv_g'],2)
    team_priors['a_gd'] = team_priors['a_off'] - team_priors['a_def']

    # Get fixtures and team data from pl api
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    team_data = pd.DataFrame(fpl_data['teams'])
    team_data = team_data[['id', 'short_name']].rename(columns={"id": "team_id"})
    team_data = team_data.replace('NFO', 'FOR')
    r = requests.get('https://fantasy.premierleague.com/api/fixtures/')
    fixtures_data = r.json()
    fixtures_data = pd.DataFrame(fixtures_data)
    fixtures_data = fixtures_data.drop('stats', axis=1)
    fixtures_data = fixtures_data[fixtures_data['started'] != True]
    # fixtures_data.to_csv('../data/fixtures_test_all.csv')
    fixtures_data = fixtures_data[fixtures_data['started'] == False]
    fixtures_data['gw'] = fixtures_data['event'].astype(int)
    # fixtures_data['datetime'] = fixtures_data['kickoff_time'].astype('datetime64[ns]')
    fixtures_data['kickoff_time'] = pd.to_datetime(fixtures_data['kickoff_time'])
    fixtures_data['datetime'] = fixtures_data['kickoff_time'].dt.tz_convert('UTC').dt.tz_localize(None)
    fixtures_data['date_str'] = fixtures_data['datetime'].dt.strftime('%Y-%m-%d')
    fixtures_data['time_str'] = fixtures_data['datetime'].dt.strftime('%H:%M')
    # fixtures_data['date_str'] = fixtures_data['kickoff_time'].str[:10]
    # fixtures_data['time_str'] = fixtures_data['kickoff_time'].str[-9:-4]
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_a', right_on='team_id', how='left').rename(columns={"short_name": "team_a_name", "team_id": "team_a_id"})
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_h', right_on='team_id', how='left').rename(columns={"short_name": "team_h_name", "team_id": "team_h_id"})

    # Add customized fixtures to fixtures table
    fixtures_data.loc[:,'customized'] = False
    fixtures_data['custom_dates'] = [[] for _ in range(len(fixtures_data))]
    fixtures_data['custom_probs'] = [[] for _ in range(len(fixtures_data))]
    if custom_fixtures is not None:
        custom_fixtures['added_to_ticker'] = False
        for index, row in custom_fixtures.iterrows():
            h = custom_fixtures.loc[index,'home_team']
            a = custom_fixtures.loc[index,'away_team']
            listy = fixtures_data.index[(fixtures_data['team_h_name'] == h) & (fixtures_data['team_a_name'] == a)].to_list()
            # if the fixture to be added isn't in the fixtures to be played, add it
            if listy != []:
                i = listy[0]
                fixtures_data.at[i, 'customized'] = True
                fixtures_data.at[i, 'custom_dates'] = custom_fixtures.loc[index,'dates']
                fixtures_data.at[i, 'custom_probs'] = custom_fixtures.loc[index,'probabilities']
                custom_fixtures.at[index, 'added_to_ticker'] = True

    # Drop rows not in gameweek range
    if gw_range is not None:
        mask = fixtures_data['gw'].isin(gw_range)
        fixtures_data = fixtures_data[mask]

    # Generate ticker
    natural_fix_dates = sorted(fixtures_data['date_str'].unique())
    custom_fix_dates = []
    if custom_fixtures is not None:
        for i, x in custom_fixtures.iterrows():
            custom_fix_dates += (custom_fixtures.loc[i, 'dates'])
    unique_dates = sorted(natural_fix_dates + custom_fix_dates)
    unique_dates = sorted(list(set(unique_dates)))
    team_fixtures = team_data.assign(**dict.fromkeys(unique_dates, ''))
    old_date = None
    old_datetime = None
    for index, row in fixtures_data.iterrows():
        new_date = row['date_str']
        new_datetime = row['datetime']
        away_team = row['team_a_name']
        home_team = row['team_h_name'].lower()
        if old_date != new_date or (row['datetime'] == old_datetime and first_fix):
            away_team += '!'
            home_team += '!'
            first_fix = True
        else:
            first_fix = False
        team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = away_team
        team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = home_team
        old_date = new_date
        old_datetime = new_datetime
    copied_natural_fixtures = team_fixtures.copy()
    # Add the custom fixtures to the ticker, deleting their 'natural' placement
    # NB: only fixtures that have yet to be played can be added
    if custom_fixtures is not None:
        for index, row in fixtures_data.iterrows():
            if row['customized']:
                team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = ''
                team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = ''
                for i, x in enumerate(row['custom_probs']):
                    if x == 1:
                        prob_str = ''
                    elif x == 0:
                        break
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['custom_dates'][i]      
                    away_team = row['team_a_name'] + '!' + prob_str
                    home_team = row['team_h_name'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date].to_list()[0] != '':
                        away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date].to_list()[0] != '':
                        home_team = '\n' + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date] += away_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date] += home_team
        # Add those fixtures which aren't included in the natural fixtures
        if False in custom_fixtures['added_to_ticker'].tolist():
            extra_custom_fixtures = custom_fixtures.loc[custom_fixtures['added_to_ticker'] == False]
            for index, row in extra_custom_fixtures.iterrows():
                for i, x in enumerate(row['probabilities']):
                    if x == 1:
                        prob_str = ''
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['dates'][i]      
                    away_team = row['away_team'] + '!' + prob_str
                    home_team = row['home_team'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list()[0] != '':
                            away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list()[0] != '':
                            home_team = '\n' + home_team
                    prev1 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] + away_team
                    prev2 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] = prev1
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] = prev2
                    custom_fixtures.loc[index, 'added_to_ticker'] = True

    # Generate matchday mapping to sky gw, fpl gw, date, and day of week
    weekdays = []
    matchdays = []
    sky_gw_list = []
    fpl_gw_list = []
    sky_gw_df = pd.read_csv('../data/sky_gw_starts_2324.csv')
    fpl_gw_df = pd.read_csv('../data/fpl_gw_starts_2324.csv')
    sky_gw_date = sky_gw_df.loc[0,'start_date']
    fpl_gw_date = fpl_gw_df.loc[0,'start_date']
    fpl_gw_index = 0
    sky_gw_index = 0
    # loop through matchdays
    for i, x in enumerate(unique_dates):
        new_day = str(datetime.strptime(unique_dates[i], '%Y-%m-%d').date().weekday())
        weekdays.append(new_day)
        matchdays.append(i+1)
        # while the date of the current matchday is later than that of the proposed sky gw, proceed to the date of the next sky gw
        while x >= sky_gw_date:
            broken = False
            if sky_gw_index > len(sky_gw_df)-1:
                broken = True
                break
            sky_gw_index += 1
            sky_gw_date = sky_gw_df.loc[sky_gw_index-1, 'start_date']
        if broken:
            sky_gw = sky_gw + 1
        else:
            sky_gw = sky_gw_df.loc[sky_gw_index-1, 'gameweek']-1
        sky_gw_list.append(sky_gw)
        # while the date of the current matchday is later than that of the proposed fpl gw, proceed to the date of the next fpl gw
        while x >= fpl_gw_date:
            broken = False
            if fpl_gw_index > len(fpl_gw_df)-1:
                broken = True
                break
            fpl_gw_index += 1
            fpl_gw_date = fpl_gw_df.loc[fpl_gw_index-1, 'start_date']
        if broken:
            fpl_gw = fpl_gw + 1
        else:
            fpl_gw = fpl_gw_df.loc[fpl_gw_index-1, 'gameweek']-1
        fpl_gw_list.append(fpl_gw)
    data = {'unique_dates': unique_dates,
            'weekday': weekdays,
            'matchday': matchdays,
            'sky_gw': sky_gw_list,
            'fpl_gw': fpl_gw_list
            }
    md_map = pd.DataFrame(data)
    date_0 = md_map.loc[0,'unique_dates']
    date_0 = datetime.strptime(date_0, '%Y-%m-%d').date()
    for i, col in md_map.iterrows():
        date_1 = md_map.loc[i,'unique_dates']
        date_1 = datetime.strptime(date_1, '%Y-%m-%d').date()
        delta = int((date_1 - date_0).days)
        md_map.loc[i,'days_elapsed'] = delta

    # Generate dataframes for all possible fixture permutations for stochastic optimization, assuming all fixtures are independent
    if generate_all_dataframes and extra_fixtures is not None:
        uncertain_fixtures = extra_fixtures.drop(extra_fixtures[extra_fixtures.probability == 1].index)
        my_list = uncertain_fixtures.probability.tolist()
        # Assume all fixtures are independent
        n_uncert_fix = len(my_list)
        number_of_permutations = 2**(n_uncert_fix)
        fix_permutation_dict = {}
        for i in range(number_of_permutations):
            fixture_key = f'permutation_{i+1}'
            permutation_string = format(i, f'0{n_uncert_fix}b')
            df = copied_natural_fixtures.copy()
            likelihood = 1
            for i, x in enumerate(permutation_string):
                if bool(int(x)):
                    away_team = uncertain_fixtures.loc[i,'away_team']
                    home_team = uncertain_fixtures.loc[i,'home_team']
                    df.loc[df['short_name'] == home_team, uncertain_fixtures.loc[i,'date']] = away_team
                    df.loc[df['short_name'] == away_team, uncertain_fixtures.loc[i,'date']] = home_team.lower()
                    likelihood = likelihood * uncertain_fixtures.loc[i,'probability']
                else:
                    likelihood = likelihood * (1-uncertain_fixtures.loc[i,'probability'])
            fix_permutation_dict[fixture_key] = {'df': df, 'likelihood': likelihood}
    else:
        fix_permutation_dict = None

    fixtures_df = team_fixtures.copy()

    # Drop excluded teams
    if exclude_teams is not None:
        for i in exclude_teams:
            team_fixtures = team_fixtures[team_fixtures.short_name != i]

    # Add gameweek superheader
    date_to_gw = fixtures_data[['date_str','gw']].drop_duplicates()
    headers = list(team_fixtures.columns.values)
    sky_gw_header = []
    fpl_gw_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'sky_gw'
            k = 'fpl_gw'
        else:
            sky_gw = md_map.loc[md_map['unique_dates']==i,'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['unique_dates']==i,'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    md_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'matchday'
        else:
            md = md_map.loc[md_map['unique_dates']==i,'matchday'].values[0]
            j = str(md)
        md_header.append(j)
    team_fixtures.columns=[sky_gw_header, fpl_gw_header, md_header, headers]


    formatted_fixtures = team_fixtures.copy()
    # Make color map dictionary and function
    color_ts = team_priors[['short_team','h_gd', 'a_gd']].copy()
    min_gd = min(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())*2.3
    max_gd = max(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())#*1.8
    norm = matplotlib.colors.Normalize(vmin=min_gd, vmax=max_gd, clip=True)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis_r)
    color_ts['h_gd_color'] = color_ts['h_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    color_ts['a_gd_color'] = color_ts['a_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    h_teams = color_ts['short_team'].values.tolist()
    a_teams = [team.lower() for team in h_teams]
    teams = h_teams + a_teams
    team_gd = color_ts['a_gd_color'].values.tolist() + color_ts['h_gd_color'].values.tolist()
    color_dict = {teams[i]: team_gd[i] for i in range(len(teams))}
    def color_col(col, pattern_map, default=''):
        return np.select(
            [col.str.contains(k, na=False) for k in pattern_map.keys()],
            [f'background-color: {v}' for v in pattern_map.values()],
            default=default
        ).astype(str)
    # Apply styles
    formatted_fixtures = formatted_fixtures.style.apply(color_col,
                                                pattern_map=color_dict
                                                , subset=team_fixtures.columns[2:]
                                                )
    formatted_fixtures = formatted_fixtures.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)
    formatted_fixtures = formatted_fixtures.set_properties(**{'color': 'white'},subset=(formatted_fixtures.columns[2:]))

    return {'formatted_fixtures': formatted_fixtures, 'fixtures_data': fixtures_df, 'matchday_map': md_map, 'fix_permutation_dict': fix_permutation_dict, 'unformatted_fixtures': team_fixtures}

## Modelling Functions

Generate dataframe of expected points values for a specified period, based on prior team and player level data. Calls fixture fixture generator function

In [3]:
def prior_team_data_gen():

    team_data = pd.read_csv('../data/team_priors.csv', index_col=0)

    return team_data

def prior_player_data_gen(team_data):
    
    prior_player_data = pd.read_csv('../data/prior_player_data.csv')
    pen_taker_override = pd.read_csv('../data/pen_taker_override.csv')
    for index, row in pen_taker_override.iterrows():
        if row['pen_share']==row['pen_share']:
            prior_player_data.loc[prior_player_data['sky_id']==row['sky_id'], 'on_pens'] = row['pen_share']
            print(str(row['reference_name']) + ' pen share overridden to ' + str(row['pen_share']))

    try:
        filepath = '../data/fplreview.csv'
        fplreview = pd.read_csv(filepath)
        fplreview = fplreview.rename(columns={'ID': 'fpl_id'})
        review_xmins = True
        print(f"Using minutes from {filepath}")
    except:
        review_xmins = False
        print(f"{filepath} not found, using default baseline minutes") 

    if review_xmins:
        # Get gw x_mins from fplreview file, and overwite
        review_gw_list = []
        for element in list(fplreview.columns.values):
            if '_xMins' in element:
                review_gw_list.append(element)
        for element in list(prior_player_data.columns.values):
            if element in review_gw_list:
                prior_player_data = prior_player_data.drop(columns = [element])
        prior_player_data = pd.merge(prior_player_data, fplreview.loc[:,['fpl_id'] + review_gw_list], on=['fpl_id'], how='inner')
    else:
        for gw in range(1,39):
            prior_player_data[str(gw)+'_xMins'] = prior_player_data['bl_xmin']

    return {'prior_player_data':prior_player_data, 'review_xmins':review_xmins}

def sky_xP_calc(sky_id, opp_team, prior_player_data, team_data, xMins, xP_breakdown=False, review_xmins=True):
    player_data = prior_player_data.loc[prior_player_data['sky_id'] == sky_id].reset_index()
    own_team = player_data.loc[0, 'short_team']
    own_team_data = team_data.loc[team_data['short_team'] == own_team].reset_index()
    opp_data = team_data.loc[team_data['short_team'] == opp_team.upper()].reset_index()
    Pos = player_data.loc[0, 'sky_pos']
    if opp_team.isupper() == True:
        home_adv = own_team_data.loc[0, 'home_adv_g']
        home_adv_pass = own_team_data.loc[0, 'home_adv_pass']
    elif opp_team.islower() == True:
        home_adv = 1/own_team_data.loc[0, 'home_adv_g']
        home_adv_pass = 1/own_team_data.loc[0, 'home_adv_pass']
    else:
        home_adv = 1
        home_adv_pass = 1
    if Pos == 'GK':
        k_G = 7
        k_CS = 7
        k_2GC = -1
    elif Pos == 'DEF':
        k_G = 7
        k_CS = 5
        k_2GC = -1
    elif Pos == 'MID':
        k_G = 6
        k_CS = 0
        k_2GC = 0
    else:
        k_G = 5
        k_CS = 0
        k_2GC = 0
    k_Start = 2
    k_Sub = 1
    k_A = 3
    k_PenSv = 5
    k_PenMiss = -3
    k_Yc = -1
    k_Rc = -3
    k_OG = -2
    k_T1 = 2
    k_T2 = 3
    x_90s = xMins/90
    if review_xmins:
        x_95s = xMins/95
        p_start = (0.5)*(0.5 + np.cbrt((x_95s-0.5)/4)) + (0.5)*x_95s
    else:
        p_start = (0.5)*(0.5 + np.cbrt((x_90s-0.5)/4)) + (0.5)*x_90s
    StartxP = k_Start * p_start
    SubxP = k_Sub * (1-p_start) * x_90s
    GxP = k_G * player_data.loc[0, 'bl_npxg'] * opp_data.loc[0, 'bl_xg_against_k'] * x_90s * home_adv * player_data.loc[0, 'fin_skill']
    AxP = k_A * player_data.loc[0, 'bl_a'] * opp_data.loc[0, 'bl_g_against_k'] * x_90s * home_adv
    OGxP = k_OG * player_data.loc[0, 'bl_og'] * opp_data.loc[0, 'bl_og_against_k'] * x_90s / home_adv
    PenScorexP = k_G * player_data.loc[0, 'on_pens'] * (player_data.loc[0, 'fin_skill'] * 0.78) * own_team_data.loc[0, 'bl_pk_att_for'] * opp_data.loc[0, 'bl_pk_att_against_k'] * x_90s * home_adv
    PenMissxP = k_PenMiss * player_data.loc[0, 'on_pens'] * (1-player_data.loc[0, 'fin_skill'] * 0.78) * own_team_data.loc[0, 'bl_pk_att_for'] * opp_data.loc[0, 'bl_pk_att_against_k'] * x_90s * home_adv
    if Pos == 'GK':
        PenSvxP = k_PenSv * own_team_data.loc[0, 'bl_pk_att_against_k'] * 0.176 * opp_data.loc[0, 'bl_pk_att_for'] * x_90s / home_adv
        mu_sv = player_data.loc[0, 'bl_sv_per_sot'] * opp_data.loc[0, 'bl_sot_for'] * own_team_data.loc[0, 'bl_sot_against_k']  * x_90s / home_adv
        T1SvxP = k_T1 * (poisson.cdf(k=4, mu=mu_sv) - poisson.cdf(k=2, mu=mu_sv))
        T2SvxP = k_T2 * (1 - poisson.cdf(k=4, mu=mu_sv))
    else:
        PenSvxP = T1SvxP = T2SvxP = 0
    YcxP = k_Yc * player_data.loc[0, 'bl_yc'] * opp_data.loc[0, 'bl_yc_against_k'] * x_90s
    RcxP = k_Rc * player_data.loc[0, 'bl_rc'] * opp_data.loc[0, 'bl_yc_against_k'] * x_90s

    mu_gc = own_team_data.loc[0, 'bl_g_against_k'] * opp_data.loc[0, 'bl_g_for'] / home_adv
    CSxP = k_CS * poisson.cdf(k=0, mu=mu_gc) * p_start * player_data.loc[0, 'bl_p_60_given_start']
    GCxP = k_2GC * ((1 - poisson.cdf(k=1, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=2, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=3, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=4, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=5, mu=mu_gc*x_90s)))

    mu_tack = player_data.loc[0, 'bl_tack'] * opp_data.loc[0, 'bl_tack_against_k'] * x_90s
    T1TackxP = k_T1 * (poisson.cdf(k=4, mu=mu_tack) - poisson.cdf(k=3, mu=mu_tack))
    T2TackxP = k_T2 * (1 - poisson.cdf(k=4, mu=mu_tack))
    av_pass = player_data.loc[0, 'bl_pass'] * opp_data.loc[0, 'bl_pass_against_k'] * x_90s * home_adv_pass
    T1PassxP = k_T1 * (norm.cdf(x=69, loc=av_pass, scale=av_pass*0.405) - norm.cdf(x=59, loc=av_pass, scale=av_pass*0.405))
    T2PassxP = k_T2 * (1 - norm.cdf(x=69, loc=av_pass, scale=av_pass*0.405))
    mu_sot = player_data.loc[0, 'bl_sot'] * opp_data.loc[0, 'bl_sot_against_k'] * x_90s * home_adv
    T1SOTxP = k_T1 * (poisson.cdf(k=2, mu=mu_sot) - poisson.cdf(k=1, mu=mu_sot))
    T2SOTxP = k_T2 * (1 - poisson.cdf(k=2, mu=mu_sot))
    xP = GxP + AxP + OGxP + PenScorexP + PenMissxP + PenSvxP + StartxP + SubxP + YcxP + RcxP + CSxP + GCxP + T1SvxP + T1TackxP + T1PassxP + T1SOTxP + T2SvxP + T2TackxP + T2PassxP + T2SOTxP
    if xP_breakdown == True:
        xP_breakdown = {'Actions': ['Start', 'Sub', 'Goal', 'Assist', 'Own Goal', 'Pen Goal', 'Pen Miss', 'Pen Save', 'Yellow Card', 'Red Card', 'Clean Sheet', 'Goal Conceded',
                                    'Tier 1 Save', 'Tier 1 Tackle', 'Tier 1 Pass', 'Tier 1 SOT', 'Tier 2 Save', 'Tier 2 Tackle', 'Tier 2 Pass', 'Tier 2 SOT', 'TOTAL'],
                        'xP': [StartxP, SubxP, GxP, AxP, OGxP, PenScorexP, PenMissxP, PenSvxP, YcxP, RcxP, CSxP, GCxP, T1SvxP, T1TackxP, T1PassxP, T1SOTxP, T2SvxP, T2TackxP, T2PassxP, T2SOTxP, xP]
                        }
        xP_breakdown = pd.DataFrame(xP_breakdown)
        xP_breakdown = xP_breakdown.drop(xP_breakdown[xP_breakdown.xP == 0].index)
        xP_breakdown.xP = round(xP_breakdown.xP,2)
    return {'xP': xP, 'xP_breakdown': xP_breakdown}

def horizontal_df_layout(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

def generate_model_output(first_md=1, last_md=14, filename_suffix=None, custom_fixtures=None, teamsheet_boost=None):
    schedule_name = 'sky_schedule'
    if filename_suffix is not None:
        schedule_name += filename_suffix
    
    if custom_fixtures is not None:
        r = generate_ticker(custom_fixtures=custom_fixtures)
    else:
        r = generate_ticker()
    md_map_2 = r['matchday_map']
    sky_schedule_2 = r['fixtures_data']
    formatted_fixtures = r['formatted_fixtures']
    unformatted_fixtures = r['unformatted_fixtures']
    headers = []
    for i, x in enumerate(sky_schedule_2.columns.values.tolist()):
        if x in md_map_2['unique_dates'].tolist():
            h = md_map_2.loc[md_map_2['unique_dates'] == x, 'matchday'].values[0]
            h = 'MD ' + str(h)
            headers.append(h)
        else:
            headers.append(x)
    sky_schedule_2.columns = headers

    if str(last_md) == last_md:
        if last_md > md_map_2['unique_dates'].tolist()[-1]:
            last_md = md_map_2.loc[len(md_map_2)-1, 'matchday']
        else:
            for i, x in enumerate(md_map_2['unique_dates'].tolist()):
                if last_md < x:
                    last_md = md_map_2.loc[i, 'matchday'] - 1
                    break
    if str(last_md) == last_md:
        last_md = md_map_2.loc[len(md_map_2)-1, 'matchday']

    matchdays = range(first_md,last_md+1)
    team_data = prior_team_data_gen()
    player_data_results = prior_player_data_gen(team_data)
    prior_player_data = player_data_results['prior_player_data']
    review_xmins = player_data_results['review_xmins']
    # prior_player_data = player_data['prior_player_data']

    fpd = pd.merge(prior_player_data, sky_schedule_2, left_on='short_team', right_on='short_name', how='left')
    
    fixture_player_data = fpd.copy()
    for i in range(first_md, last_md+1):
        gw = md_map_2.loc[md_map_2['matchday']==i, 'fpl_gw'].values[0]
        if f'{gw}_xMins' not in fixture_player_data.columns:
            if f'{gw-1}_xMins' not in fixture_player_data.columns:
                fixture_player_data[f'{gw}_xMins'] = fixture_player_data[f'{gw+1}_xMins']
            else:
                fixture_player_data[f'{gw}_xMins'] = fixture_player_data[f'{gw-1}_xMins']
        fixture_player_data[f'MD {i} Game'] = fixture_player_data[f'MD {i}'].str.len() > 1.5
        fixture_player_data[f'MD_{i}_xMins'] = fixture_player_data[f'{gw}_xMins'] * fixture_player_data[f'MD {i} Game']
    players = fixture_player_data.index.tolist()
    for p in players:
        SKY_ID = fixture_player_data.loc[p, 'sky_id']
        for m in matchdays:
            xMins = fixture_player_data.loc[p, f'MD_{m}_xMins']
            if xMins < 5:
                xP = 0
            else:
                fix_string = fixture_player_data.loc[p, f'MD {m}']
                if '\n' in fix_string:
                    xP = 0
                    fix_list = fix_string.split('\n')
                    for i, x in enumerate(fix_list):
                        r = sky_xP_calc(SKY_ID, x[:3], fixture_player_data, team_data, xMins, xP_breakdown=False, review_xmins=review_xmins)
                        sub_xP = r['xP']
                        if any(c.isdigit() for c in x):
                            sub_xP = sub_xP * int(''.join(filter(str.isdigit, x))) / 100
                        xP += sub_xP
                else:
                    r = sky_xP_calc(SKY_ID, fixture_player_data.loc[p, f'MD {m}'][:3], fixture_player_data, team_data, xMins, xP_breakdown=False)
                    xP = r['xP']
                    if any(c.isdigit() for c in fix_string):
                        xP = xP * int(''.join(filter(str.isdigit, fix_string))) / 100
                if teamsheet_boost is not None:
                    if '!' in fix_string:
                        xP = xP * (1+teamsheet_boost)
            fixture_player_data.loc[p, f'MD_{m}_Pts'] = round(xP, 2)
    skymodel_output = pd.concat([fixture_player_data.loc[:,['sky_id', 'fbref_player', 'short_team', 'sky_pos', 'sky_value']],
                                    fixture_player_data.iloc[:,-(last_md-first_md+1):]],axis = 1)
    skymodel_output['Total_Pts'] = skymodel_output.iloc[:, -(last_md-first_md+1):].sum(axis=1)
    skymodel_output = skymodel_output.fillna(0)
    filename = 'skymodel_output'
    if filename_suffix is not None:
        filename += filename_suffix
    skymodel_output.to_csv(f'../data/{filename}.csv')
    md_map_2.to_csv(f'../data/md_map.csv', index = False)

    # archive model outputs
    add_to_archive = True
    gw = md_map_2.loc[md_map_2['matchday']==first_md, 'fpl_gw'].values[0]
    md_map_filepath = f'../data/past_model_outputs/md_map_fplgw{gw}.csv'
    my_file = Path(md_map_filepath)
    if my_file.is_file():
        md_map_old = pd.read_csv(md_map_filepath)
        md_map_2['weekday'] = md_map_2['weekday'].astype('int')
        if not md_map_old.iloc[0].equals(md_map_2.iloc[0]):
            add_to_archive = False
            print('Note: model outputs generated mid gameweek will not be archived')

    if add_to_archive:
        skymodel_output.to_csv(f'../data/past_model_outputs/{filename}_fplgw{gw}.csv')
        md_map_2.to_csv(f'../data/past_model_outputs/md_map_fplgw{gw}.csv', index = False)
        filepath = '../data/fplreview.csv'
        my_file = Path(filepath)
        if my_file.is_file():
            fplreview = pd.read_csv(filepath)
            fplreview = fplreview.rename(columns={'ID': 'fpl_id'})
            for i in range(40):
                if f'{i}_Pts' in fplreview.columns.tolist():
                    fplreview = fplreview.drop(f'{i}_Pts', axis=1)
            fplreview.to_csv(f'../data/past_model_outputs/review_xmins_fplgw{gw}.csv', index = False)
        # prior_player_data.to_csv(f'../data/past_model_outputs/prior_player_data_fplgw{gw}.csv', index = False)
        # team_data.to_csv(f'../data/past_model_outputs/team_data_fplgw{gw}.csv', index = False)
        print('Model outputs archived')

    return {'skymodel_output':skymodel_output, 'formatted_fixtures':formatted_fixtures, 'md_map': md_map_2, 'unformatted_fixtures': unformatted_fixtures}

def read_skymodel_output(max_ev_cutoff=0.3, max_ev_per_price_cutoff=0.3, initial_squad=None, flatten_final_matchdays=None):
    skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('sky_id').fillna(0)

    skymodel_output['ev_per_price'] = skymodel_output['Total_Pts'] / skymodel_output['sky_value']
    
    skymodel_output_unfiltered = skymodel_output.copy()

    if initial_squad is None:
        initial_squad = []

    ev_cutoff = skymodel_output['Total_Pts'].max() * max_ev_cutoff
    skymodel_output = skymodel_output[skymodel_output['Total_Pts'] > ev_cutoff]
    
    ev_per_price_cutoff = skymodel_output['ev_per_price'].max() * max_ev_per_price_cutoff
    skymodel_output = skymodel_output[skymodel_output['ev_per_price'] > ev_per_price_cutoff]
    removed_players = [i for i in initial_squad if i not in skymodel_output.index.values.tolist()]

    for id in removed_players:
        player_row = skymodel_output_unfiltered.loc[id, :].values.flatten().tolist()
        skymodel_output.loc[id] = player_row

    md_map = pd.read_csv('../data/md_map.csv')

    return skymodel_output, md_map

def read_fpl_kid_model(max_ev_cutoff=0.3, max_ev_per_price_cutoff=0.3):
    filepath = '../data/fplkid.csv'
    fpl_kid = pd.read_csv(filepath).fillna(0)
    fpl_kid = fpl_kid.rename(columns = {'Pos':'sky_pos', 'ID':'sky_id', 'BV':'sky_value'})
    fpl_kid['Total_Pts'] = 0
    for i in range(200):
        fpl_kid = fpl_kid.rename(columns = {f'GD{i}_Pts':f'MD_{i}_Pts',f'GD{i}_xMins':f'MD_{i}_xMins'})
        if f'MD_{i}_Pts' in fpl_kid.columns.tolist():
            fpl_kid['Total_Pts'] += fpl_kid[f'MD_{i}_Pts']
    fpl_kid = fpl_kid.drop(columns=['Name', 'SV', 'Team'])

    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('G','GK')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('F','FOR')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('D','DEF')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('M','MID')

    player_data_merge = pd.read_csv('../data/prior_player_data.csv')
    player_data_merge = player_data_merge[['sky_id','fbref_player','short_team']]

    fpl_kid = pd.merge(left = fpl_kid, right = player_data_merge, on='sky_id', how='inner')
    fpl_kid = fpl_kid.set_index('sky_id')
    skymodel_output = fpl_kid.copy()
    
    ev_cutoff = skymodel_output['Total_Pts'].max() * max_ev_cutoff
    skymodel_output = skymodel_output[skymodel_output['Total_Pts'] > ev_cutoff]

    skymodel_output['ev_per_price'] = skymodel_output['Total_Pts'] / skymodel_output['sky_value']
    ev_per_price_cutoff = skymodel_output['ev_per_price'].max() * max_ev_per_price_cutoff
    skymodel_output = skymodel_output[skymodel_output['ev_per_price'] > ev_per_price_cutoff]

    md_map = pd.read_csv('../data/md_map.csv')

    return skymodel_output, md_map

def generate_cap_matrix(last_md = 20, max_ev_diff = 1):
    # Read the data from the CSV file
    skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('sky_id').fillna(0)

    # Define the maximum matchday number
    md = last_md

    # Create a DataFrame to store the results
    result_data = []

    # Generate values from 0.0 to -1.0 in 0.1 increments
    distance_to_cap_values = np.arange(0.0, -max_ev_diff-0.1, -0.1)

    # Iterate through the distance_to_cap values
    for distance_to_cap in distance_to_cap_values:
        # Round the distance_to_cap value to the nearest 0.1
        rounded_distance_to_cap = round(distance_to_cap, 1)
        
        # Create a list to store players for each matchday
        players_list = []
        
        # Iterate through each matchday column
        for md_num in range(1, md + 1):
            # Calculate the maximum score in the current matchday column for all players
            max_score = skymodel_output[f'MD_{md_num}_Pts'].max()
            
            # Calculate the difference between each player's score and the maximum score
            skymodel_output[f'MD_{md_num}_Cap'] = skymodel_output[f'MD_{md_num}_Pts'] - max_score
            
            # Filter players whose MD_{md}_Cap value matches the rounded_distance_to_cap
            players = skymodel_output[skymodel_output[f'MD_{md_num}_Cap'].round(1) == rounded_distance_to_cap]['fbref_player'].tolist()
            
            # Store the players for the current gameweek in the players_list
            players_list.append(players)

        # display(players_list)
        new_list = []
        for p in players_list:
            if len(p) == 0:
                new_list.append([])
            else:
                sublist = []
                for q in p:
                    sublist.append(q.split(' ')[-1])
                new_list.append(sublist)
        players_list = new_list
        # display(players_list)

        # Append the row data to the result_data list
        result_data.append([rounded_distance_to_cap] + players_list)
    # Create the result DataFrame
    column_headings = ['distance_to_cap'] + [f'MD_{md_num}' for md_num in range(1, md + 1)]
    cap_matrix = pd.DataFrame(result_data, columns=column_headings)

    # Format the player names without square brackets and inverted commas
    cap_matrix = cap_matrix.applymap(lambda x: '\n'.join(x) if isinstance(x, list) else x)

    # date_to_gw = fixtures_data[['date_str','gw']].drop_duplicates()
    md_map = pd.read_csv('../data/md_map.csv')
    headers = list(cap_matrix.columns.values)
    sky_gw_header = []
    fpl_gw_header = []
    for i in headers:
        # display(i)
        if i.startswith('MD_') == False:
            j = 'sky_gw'
            k = 'fpl_gw'
        else:
            matchday = int(i.split('_')[1])
            sky_gw = md_map.loc[md_map['matchday']==matchday,'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==matchday,'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    cap_matrix.columns=[sky_gw_header, fpl_gw_header, headers]

    cap_matrix_data = cap_matrix.copy()
    
    d = dict(selector="th", props=[('text-align', 'center')])
    cap_matrix = cap_matrix.style.set_properties(**{'height':'4em', 'width':'4em', 'text-align':'center'})\
        .set_table_styles([d])


    def highlight_cols(s, props=''):
        return np.where(s!='', props, '')
    MDs = [col for col in cap_matrix_data.columns if 'MD_' in col]
    # display(cap_matrix_data.columns)
    for index, row in cap_matrix_data.iterrows():
        rgb_index = index / max_ev_diff
        rgb_str = f'rgb({10+15*rgb_index},{10*rgb_index},{40+14*rgb_index})'
        cap_matrix = cap_matrix.apply(highlight_cols, props="background-color: "+rgb_str, subset=([index], cap_matrix.columns[1:]))


    cap_matrix = cap_matrix.set_properties(**{'color': 'white'},subset=(cap_matrix.columns[1:]))

    cap_matrix = cap_matrix.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)

    cap_matrix = cap_matrix.format(precision=1)
    # Export the resulting table as a CSV file in the data folder
    #result_df.to_csv('../data/sky_caps_matrix.csv', index=False)
    return cap_matrix

## Optimization Functions

Generate optimal solution for team or analyze multiple simulated runs with noise, based on the model output

In [4]:
# Generate optimal plan
data = {'sky_pos': ['GK', 'DEF', 'MID', 'FOR'],
        'squad_min_play': [1, 3, 3, 1],
        'squad_max_play': [1, 5 ,5, 3]}
type_data = pd.DataFrame(data, index=[1,2,3,4])

def solve_sky_mp(initial_squad, input_data, md_map, next_md=1, last_md=10, flatten_mds_after=None,
                 ta_tot=50, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=7.5, 
                 vicecap_wt = 0.05,
                 exclusions=None, keeps=None, ban_teams=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                 apply_noise=False, seed_val=None, magnitude=1, show_itb=False, show_non_team_ev=False):
    
    if decay_base >= 1 or decay_base <=0:
        decay_base = 1
        objective = 'regular'

    def string_day_to_int(input_string):
        if str(input_string) == input_string:
            if input_string > md_map['unique_dates'].tolist()[-1]:
                input_string = md_map.loc[len(md_map)-1, 'matchday']
            else:
                for i, x in enumerate(md_map['unique_dates'].tolist()):
                    if input_string < x:
                        input_string = md_map.loc[i, 'matchday'] - 1
                        break
        if str(input_string) == input_string:
            input_string = md_map.loc[len(md_map)-1, 'matchday']
        return input_string

    last_md = string_day_to_int(last_md)

    if flatten_mds_after is not None:
        flatten_mds_after = string_day_to_int(flatten_mds_after)
        for matchday in range(flatten_mds_after, last_md):
            input_data[f'MD_{flatten_mds_after}_Pts'] += input_data[f'MD_{matchday+1}_Pts'] * pow(decay_base, matchday+1-flatten_mds_after)
            input_data = input_data.drop(f'MD_{matchday+1}_Pts', axis=1)
        last_md = flatten_mds_after
        
    horizon = last_md + 1 - next_md
    problem_name = f'sky_mp_h{horizon}_d1'
    
    # Sets
    players = input_data.index.tolist()
    element_types = type_data.index.tolist()
    matchdays = list(range(next_md, next_md+horizon))
    cap_mds = [i for i in matchdays if i != flatten_mds_after]
    all_md = [next_md-1] + matchdays
    
    first_gw = int(md_map.loc[md_map['matchday']==next_md, 'sky_gw'].values[0])
    last_gw = int(md_map.loc[md_map['matchday']==last_md, 'sky_gw'].values[0])
    gameweeks = list(range(first_gw,last_gw+1))
    gw_transfer_allowance = {w: 5 for w in gameweeks}
    gw_transfer_allowance[first_gw] = min(ta_gw,5)

    if apply_noise:
        rng = np.random.default_rng(seed = seed_val)
        input_data['Total_Pts'] = 0
        player_df = pd.read_csv('../data/prior_player_data.csv')
        player_df = player_df[['sky_id', 'noise_factor']]
        input_data = pd.merge(input_data,player_df,on='sky_id').set_index('sky_id')
        for m in matchdays:
            # replaced `input_data['noise_factor']` with 1.5 for now
            noise = input_data[f'MD_{m}_Pts'] * 0.035 * rng.standard_normal(size = len(input_data)) * magnitude * 1.5 
            input_data[f'MD_{m}_Pts'] = input_data[f'MD_{m}_Pts'] + round(noise,2)
            input_data['Total_Pts'] += input_data[f'MD_{m}_Pts']

    # Model
    model = so.Model(name = 'multi_period')

    # Variables
    squad = model.add_variables(players, all_md, name='squad', vartype=so.binary)
    captain = model.add_variables(players, matchdays, name='captain', vartype=so.binary)
    vicecap = model.add_variables(players, matchdays, name='vicecap', vartype=so.binary)
    transfer_in = model.add_variables(players, matchdays, name='transfer_in', vartype=so.binary)
    transfer_out = model.add_variables(players, matchdays, name='transfer_out', vartype=so.binary)
    
    # Dictionaries
    squad_type_count = {(t,d): so.expr_sum(squad[p,d] for p in players if input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']) for t in element_types for d in matchdays}
    player_value = (input_data['sky_value']).to_dict()
    # bought_amount = {d: so.expr_sum(player_value[p] * transfer_in[p,d] for p in players) for d in matchdays}
    # sold_amount = {d: so.expr_sum(player_value[p] * transfer_out[p,d] for p in players) for d in matchdays}
    squad_value = {d: so.expr_sum(player_value[p] * squad[p,d] for p in players) for d in matchdays}
    points_player_day = {(p,d): input_data.loc[p, f'MD_{d}_Pts'] for p in players for d in matchdays}
    squad_count = {d: so.expr_sum(squad[p, d] for p in players) for d in matchdays}
    
    total_number_of_transfers = so.expr_sum(transfer_out[p,d] for p in players for d in matchdays) 

    md_number_of_transfers = {d: so.expr_sum(transfer_out[p,d] for p in players) for d in matchdays}        
    gw_number_of_transfers = {w: so.expr_sum(md_number_of_transfers[d] for d in matchdays if int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0]) == w) for w in gameweeks}
    
    # Initial Conditions
    if initial_squad is not None:
        model.add_constraints((squad[p, next_md-1] == 1 for p in initial_squad), name='initial_squad_players')
        model.add_constraints((squad[p, next_md-1] == 0 for p in players if p not in initial_squad), name='initial_squad_others')
    # Constraints: squad and captaincy
    model.add_constraints((squad_count[d] == 11 for d in matchdays), name='squad_count')
    model.add_constraints((so.expr_sum(captain[p,d] for p in players) == 1 for d in cap_mds), name='captain_count')
    model.add_constraints((captain[p,d] <= squad[p,d] for p in players for d in matchdays), name='captain_squad_rel')
    if vicecap_wt > 0 and vicecap_wt < 1:
        model.add_constraints((so.expr_sum(vicecap[p,d] for p in players) == 1 for d in cap_mds), name='vicecap_count')
        model.add_constraints((vicecap[p,d] <= squad[p,d] for p in players for d in matchdays), name='vicecap_squad_rel')
        model.add_constraints((captain[p,d] + vicecap[p,d] <=1 for p in players for d in matchdays), name='cap_vicecap_rel')
    else:
        model.add_constraints((so.expr_sum(vicecap[p,d] for p in players) == 0 for d in cap_mds), name='no_vicecap')
        vicecap_wt = 0
    if flatten_mds_after is not None:
        model.add_constraints((captain[p,flatten_mds_after] + vicecap[p,flatten_mds_after] == 0 for p in players), name='no_cap_flattened_md')
    # Constraints: formation and budget
    model.add_constraints((squad_type_count[t,d] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']] for t in element_types for d in matchdays), name='valid_formation_1')
    model.add_constraints((squad_type_count[2,d]-squad_type_count[4,d] <= 3.5 for d in matchdays), name='valid_formation_2')
    model.add_constraints((squad_value[d] <= 100 for d in matchdays), name='squad_budget')
    # Constraints: transfers
    model.add_constraints((squad[p,d] == squad[p,d-1] + transfer_in[p,d] - transfer_out[p,d] for p in players for d in matchdays), name='squad_transfer_rel')
    model.add_constraint(total_number_of_transfers <= min(ta_tot,50), name = 'transfer_allowance')
    model.add_constraints((gw_number_of_transfers[w] <= gw_transfer_allowance[w] for w in gameweeks), name = 'gw_transfer_allowance')
    if no_transfer_mds is not None:
        model.add_constraints((md_number_of_transfers[m] == 0 for m in no_transfer_mds), name='no_transfer_matchdays')
    # Constraints: specified players
    # Ban teams
    if ban_teams is not None:
        if exclusions is None:
            exclusions = []
        for index, row in input_data.iterrows():
            if row['short_team'] in ban_teams:
                exclusions += [index]
    # Force Exclude
    if exclusions is not None:
        model.add_constraints((squad[e, d] == 0 for e in exclusions for d in matchdays), name = 'force_exclude_players')
    # Force Keep
    if keeps is not None:
        model.add_constraints((squad[e, d] == 1 for e in keeps for d in matchdays), name = 'force_keep_players')
    # Force transfer in
    if force_transfer_in is not None:
        fti_any = []
        fti_spec = []
        for constraint_dict in force_transfer_in:
            if "md" not in constraint_dict:
                fti_any.append(constraint_dict)
            else:
                fti_spec.append(constraint_dict)
        if len(fti_any) > 0.5:
            # suppress singular constraint warning message 
            with warnings.catch_warnings():
                warnings.simplefilter("ignore") 
                model.add_constraints((so.expr_sum(transfer_in[fti_any[e]["sky_id"], d] for e in list(range(len(fti_any))) for d in matchdays) >= 1), name='force_transfer_in_any')
        if len(fti_spec) > 0.5:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.add_constraints((transfer_in[fti_spec[e]["sky_id"], fti_spec[e]["md"]] == 1 for e in list(range(len(fti_spec)))), name = 'force_transfer_in_specified')
                model.add_constraints((transfer_out[fti_spec[e]["sky_id"], fti_spec[e]["md"]] == 0 for e in list(range(len(fti_spec)))), name = 'force_transfer_in_specified_lock')
    # Force transfer out
    if force_transfer_out is not None:
        fto_any = []
        fto_spec = []
        for constraint_dict in force_transfer_out:
            if "md" not in constraint_dict:
                fto_any.append(constraint_dict)
            else:
                fto_spec.append(constraint_dict)
        if len(fto_any) > 0.5:
            # suppress singular constraint warning message 
            with warnings.catch_warnings():
                warnings.simplefilter("ignore") 
                model.add_constraints((so.expr_sum(transfer_out[fto_any[e]["sky_id"], d] for e in list(range(len(fto_any))) for d in matchdays) >= 1), name='force_transfer_out_any')
        if len(fto_spec) > 0.5:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.add_constraints((transfer_out[fto_spec[e]["sky_id"], fto_spec[e]["md"]] == 1 for e in list(range(len(fto_spec)))), name = 'force_transfer_out_specified')
                model.add_constraints((transfer_in[fto_spec[e]["sky_id"], fto_spec[e]["md"]] == 0 for e in list(range(len(fto_spec)))), name = 'force_transfer_out_specified_lock')

    # Objective
    md_xp = {d: so.expr_sum(points_player_day[p,d] * (squad[p,d] + captain[p,d] + vicecap_wt*vicecap[p,d]) for p in players) for d in matchdays}
    if objective == 'regular':
        # Note `- 0.0001*md_number_of_transfers[d]*d` gives a negligible incentive to make a transfer as late as possible
        eval_score = so.expr_sum(md_xp[d] + 0.0001*md_number_of_transfers[d]*d for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-eval_score, sense='N', name='total_regular_xp') 
    else:
        # eval_score = so.expr_sum(md_xp[d] * pow(decay_base, d-next_md) for d in matchdays) - total_number_of_transfers*transfer_cost
        days_elapsed0 = md_map.loc[md_map['matchday']==next_md,'days_elapsed'].values[0]
        # Convert weekly decay to daily
        decay_base = decay_base ** (1/7)
        eval_score = so.expr_sum(md_xp[d] * pow(decay_base, md_map.loc[md_map['matchday']==d,'days_elapsed'].values[0]-days_elapsed0) + 0.0001*md_number_of_transfers[d]*d for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-eval_score, sense='N', name='total_decay_xp')
    
    # Solve Step
    model.export_mps(filename='skyoutput.mps')
    command = f'cbc skyoutput.mps solve solu {problem_name}_sol.txt'
    # !{command}
    os.system(command)
    # Read the solution back to the file
    with open(f'{problem_name}_sol.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
            
    # (OLD) Generate a dataframe to display the solution 
    picks = []
    for d in matchdays:
        for p in players:
            if squad[p,d].get_value() + transfer_out[p,d].get_value() > 0.5:
                lp = input_data.loc[p]
                is_captain = 1 if captain[p,d].get_value() > 0.5 else 0
                is_transfer_in = 1 if transfer_in[p,d].get_value() > 0.5 else 0
                is_transfer_out = 1 if transfer_out[p,d].get_value() > 0.5 else 0
                picks.append([
                    int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0]), d, lp['fbref_player'], lp['sky_pos'], lp['short_team'], lp['sky_value'], round(points_player_day[p,d], 2), is_captain, is_transfer_in, is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns=['sky_gw','matchday','name', 'pos', 'team', 'value', 'xP', 'captain', 'transfer_in', 'transfer_out'])#.sort_values(by=['matchday'])
    picks_df.loc[picks_df['matchday'] == next_md, 'transfer_in'] = 0
    
    eval_score = round(eval_score.get_value(),2)
    total_xp = round(so.expr_sum(points_player_day[p,d] * (squad[p,d] + captain[p,d]) for p in players for d in matchdays).get_value(), 2)
    
    # Generate a better dataframe to display the solution
    plan = []
    for t in element_types:
        for p in players:
            if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']:
                lp = input_data.loc[p]
                player_info = [p, lp['short_team'], lp['sky_pos'][0], lp['sky_value'], lp['fbref_player']]
                for d in matchdays:
                    current_points = points_player_day[p,d]
                    if squad[p,d].get_value() > 0.5:
                        score = f'{round(current_points, 2)}'
                        if captain[p,d].get_value() > 0.5:
                            score += 'c'
                        if vicecap[p,d].get_value() > 0.5:
                            score += 'v'
                    elif show_non_team_ev and current_points > 0.2:
                        score = f'({round(current_points, 2)})'
                    else:
                        score = ''
                    player_info.append(score)
                plan.append(player_info)
    columns = ['ID','Team', 'Pos','Value','Name']
    for d in matchdays:
        # w = int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0])
        columns.append(f"{d}")
    plan_df = pd.DataFrame(plan, columns=columns)
    plan_df = plan_df.replace(['0.0'],'-')
    plan_df = plan_df.replace(['0.0c'],'-')
    plan_df = plan_df.replace(['0.0v'],'-')

    if show_itb:
        itb_row = ['','','','','£ITBm']
        for d in matchdays:
            itb = abs(100 - squad_value[d].get_value())
            itb_row.append(itb)
        plan_df.loc[len(plan_df)] = itb_row

    # make dataframe to record the players in a simulation
    plan = []
    if apply_noise:
        for t in element_types:
            for p in players:
                if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']:
                    lp = input_data.loc[p]
                    player_info = [p, lp['short_team'], lp['sky_pos'][0], lp['sky_value'], lp['fbref_player']]
                    for d in matchdays:
                        if squad[p,d].get_value() > 0.5:
                            score = 1
                        else:
                            score = 0
                        player_info.append(score)
                    plan.append(player_info)
        columns = ['ID','Team', 'Pos','Value','Name']
        for d in matchdays:
            # w = int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0])
            columns.append(f"{d}")
        players_in_sim = pd.DataFrame(plan, columns=columns)
    else:
        players_in_sim = None
    
    sky_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'sky_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            sky_gw = md_map.loc[md_map['matchday']==int(i),'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==int(i),'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    plan_df.columns=[sky_gw_header, fpl_gw_header, columns]

    transfers_made = int(total_number_of_transfers.get_value())
    
    return{'model': model, 'picks': picks_df, 'total_xp': total_xp, 'eval_score': eval_score, 'plan': plan_df, 'transfers_made': transfers_made, 'players_in_sim': players_in_sim}

In [5]:
# Produce sensitivity analysis with noise
def solve_sky_mp_noise(initial_squad, input_data, md_map, next_md=1, last_md=10, flatten_mds_after=None,
                       ta_tot=50, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=7.5, 
                       vicecap_wt = 0.05,
                       exclusions=None, keeps=None, ban_teams=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                       show_itb=False, show_non_team_ev=False,
                       seed_val=None, nsims=5, magnitude=1):
    transfer_sum = 0
    xp_sum = 0
    eval_sum = 0

    baseline_projection_df = input_data

    results_dict = {}

    for i in range(nsims):
        print(f"Running sim {i+1} of {nsims}...")
        input_data = baseline_projection_df.copy()
        results = solve_sky_mp(initial_squad=initial_squad, input_data=input_data, md_map=md_map, next_md=next_md, last_md=last_md, flatten_mds_after=flatten_mds_after,
                    ta_tot=ta_tot, ta_gw=ta_gw, objective=objective, decay_base=decay_base, transfer_cost=transfer_cost,
                    exclusions=exclusions, keeps=keeps, ban_teams=ban_teams, force_transfer_in=force_transfer_in, force_transfer_out=force_transfer_out, no_transfer_mds=no_transfer_mds,
                    show_itb=show_itb, show_non_team_ev=show_non_team_ev,
                    apply_noise=True, seed_val=seed_val, magnitude=magnitude)
        results_dict[i] = results['plan']
        players_in_sim = results['players_in_sim']
        if i == 0:
            sensitivity_df = players_in_sim
        else:
            for index, row in results['players_in_sim'].iterrows():
                if row['ID'] in sensitivity_df['ID'].tolist():
                    sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] = sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] + row['1':]
                    continue
                else:
                    row_to_append = row.tolist()
                    sensitivity_df.loc[len(sensitivity_df)] = row_to_append

        clear_output(wait=True)
        transfer_sum += results['transfers_made']
        xp_sum += results['total_xp']
        eval_sum += results['eval_score']
    avg_trf = round(transfer_sum/nsims,2)
    avg_xp = round(xp_sum/nsims,2)
    avg_eval = round(eval_sum/nsims,2)
    # display(sensitivity_df)
    # sensitivity_df.loc[:,'1':] = sensitivity_df.loc[:,'1':].astype(int)
    sensitivity_df.loc[:,'1':] = sensitivity_df.loc[:,'1':] * 100 / nsims
    # sensitivity_df.loc[:,'1':] = round(sensitivity_df.loc[:,'1':],0)
    sensitivity_df.loc[:,'1':] = sensitivity_df.loc[:,'1':].astype(int)

    # Sort the dataframe by initial team and position
    sensitivity_df['max_ocuurences'] = 0
    sensitivity_df['init_team'] = 0
    sensitivity_df['pos_code'] = 0
    for index, row in sensitivity_df.iterrows():
        if row['Pos'] == 'G':
            sensitivity_df.loc[index,'pos_code'] = 1
        elif row['Pos'] == 'D':
            sensitivity_df.loc[index,'pos_code'] = 2
        elif row['Pos'] == 'M':
            sensitivity_df.loc[index,'pos_code'] = 3
        else:
            sensitivity_df.loc[index,'pos_code'] = 4
        if row['ID'] in initial_squad:
            sensitivity_df.loc[index,'init_team'] = 1
    sensitivity_df = sensitivity_df.sort_values(by=['pos_code', 'init_team'], ascending=[True, False])
    sensitivity_df.drop(['max_ocuurences', 'init_team', 'pos_code'], axis=1, inplace=True)
    
    columns = sensitivity_df.columns.values.tolist()

    sky_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'sky_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            sky_gw = md_map.loc[md_map['matchday']==int(i),'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==int(i),'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    sensitivity_df.columns=[sky_gw_header, fpl_gw_header, columns]

    sens = sensitivity_df.copy()
    sens = sens.style.background_gradient(cmap="RdPu", subset=sensitivity_df.columns[5:]).format(precision=1)
    # sens.set_properties(**{'text-align': 'left'})
    sens = sens.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)
                
    return {'sensitivity_df': sens, 'avg_trf': avg_trf, 'avg_xp': avg_xp, 'avg_eval': avg_eval, 'sensitivity_df_unformatted': sensitivity_df, 'results_dict': results_dict}

## Solver Commands
Commands to generate EV and optimal plans

In [6]:
# To generate player EV from the fixtures:
# Change player or team underlyings as you see fit in prior_player_data.csv and team_priors.csv
# Penalty takers can be overridden in pen_taker_override.csv
# Adding an fplreview.csv to the data folder will let the model use its xMins. Alternatively, edit bl_xmins in prior_player_data.csv to manually override them 

# Example custom fixtures dataframe to add, move, or remove fixtures and assign probabilities of occuring
# Leave 'custom_fixtures=None' to keep the fixtures as they are on the PL site
df = pd.DataFrame(columns=('home_team', 'away_team', 'dates', 'probabilities'))
df.loc[len(df)] = ['LIV', 'LUT', ['2024-02-20', '2024-02-24', '2024-04-23'], [0.5, 0.22, 0.28]]
df.loc[len(df)] = ['BOU', 'LUT', ['2024-02-20', '2024-03-12'], [0.5, 0.5]]
df.loc[len(df)] = ['MCI', 'BRE', ['2024-02-20', '2024-03-12', '2024-04-23'], [0.6, 0.25, 0.15]]

df.loc[len(df)] = ['WHU', 'BRE', ['2024-02-25', '2024-02-26'], [0.7, 0.3]]

df.loc[len(df)] = ['MUN', 'FUL', ['2024-02-20', '2024-04-23'], [0.81, 0.19]]
df.loc[len(df)] = ['CHE', 'TOT', ['2024-02-20', '2024-04-23'], [0.2, 0.8]]

df.loc[len(df)] = ['WHU', 'AVL', ['2024-03-16', '2024-04-23'], [0.53, 0.47]]
df.loc[len(df)] = ['MUN', 'SHU', ['2024-03-16', '2024-04-23'], [0.36, 0.64]]
df.loc[len(df)] = ['FUL', 'TOT', ['2024-03-16', '2024-04-23'], [0.58, 0.42]]
df.loc[len(df)] = ['EVE', 'LIV', ['2024-03-16', '2024-04-23'], [0.2, 0.8]]
df.loc[len(df)] = ['CRY', 'NEW', ['2024-03-16', '2024-04-23'], [0.56, 0.44]]
df.loc[len(df)] = ['BHA', 'MCI', ['2024-03-16', '2024-04-23'], [0.28, 0.72]]
df.loc[len(df)] = ['ARS', 'CHE', ['2024-03-16', '2024-04-23'], [0.63, 0.37]]
df.loc[len(df)] = ['LUT', 'FOR', ['2024-03-16', '2024-04-23'], [0.69, 0.31]]
df.loc[len(df)] = ['BUR', 'BRE', ['2024-03-16', '2024-04-23'], [0.88, 0.12]]
df.loc[len(df)] = ['WOL', 'BOU', ['2024-03-16', '2024-04-23'], [0.52, 0.48]]

# Generate fixture ticker and player EV
# Setting "teamsheet_boost" to a positive decimal (e.g. 0.05) will boost the EV of players for whom the teamsheet will be released before the deadline
# Set the last matchday, last_md, as an integer, or a date as a string in the format 'YYYY-mm-dd'
r2 = generate_model_output(first_md=1, last_md='2024-04-23', filename_suffix=None, custom_fixtures=df, teamsheet_boost=0.05)
display(r2['formatted_fixtures'])
# display(r2['unformatted_fixtures'])
display(r2['skymodel_output'].sort_values(by=['Total_Pts'], ascending=False).head(20))

Using minutes from ../data/fplreview.csv
Note: model outputs generated mid gameweek will not be archived


,sky_id,fbref_player,short_team,sky_pos,sky_value,MD_1_Pts,MD_2_Pts,MD_3_Pts,MD_4_Pts,MD_5_Pts,MD_6_Pts,MD_7_Pts,MD_8_Pts,MD_9_Pts,MD_10_Pts,MD_11_Pts,MD_12_Pts,MD_13_Pts,MD_14_Pts,MD_15_Pts,MD_16_Pts,MD_17_Pts,MD_18_Pts,MD_19_Pts,MD_20_Pts,MD_21_Pts,MD_22_Pts,MD_23_Pts,MD_24_Pts,MD_25_Pts,MD_26_Pts,MD_27_Pts,MD_28_Pts,MD_29_Pts,MD_30_Pts,MD_31_Pts,MD_32_Pts,MD_33_Pts,MD_34_Pts,Total_Pts
0,1280,Erling Haaland,MCI,FOR,12.0,0.0,2.85,0.00,0.00,0.00,0.00,0.00,9.31,0.00,0.00,0.00,7.16,8.43,0.00,0.0,7.32,0.00,0.0,4.77,0.0,7.36,0.00,0.00,7.94,5.98,1.94,1.83,6.35,0.00,6.88,6.66,9.24,7.15,5.92,107.09
13,398,Rodri,MCI,MID,9.4,0.0,5.54,0.00,0.00,0.00,0.00,0.00,6.67,0.00,0.00,0.00,5.93,6.68,0.00,0.0,5.89,0.00,0.0,3.68,0.0,6.05,0.00,0.00,6.29,4.49,1.51,1.57,5.14,0.00,4.91,5.90,6.62,5.17,4.77,86.81
24,369,Virgil van Dijk,LIV,DEF,9.9,0.0,0.00,0.00,0.00,6.06,0.00,0.00,5.24,0.00,0.00,3.55,0.00,6.69,0.00,0.0,5.27,0.00,0.0,3.76,0.0,1.61,0.00,0.00,6.32,3.89,0.00,1.21,5.31,0.00,7.28,4.98,6.70,5.74,6.88,80.49
16,1247,Dominic Solanke,BOU,FOR,7.8,0.0,0.00,0.00,0.00,5.26,0.00,0.00,0.00,5.36,0.00,6.59,0.00,4.93,0.00,0.0,4.15,0.00,0.0,3.29,0.0,4.01,0.00,0.00,5.12,6.67,3.29,2.68,5.82,5.46,0.00,5.32,5.60,4.34,2.45,80.34
7,337,Trent Alexander-Arnold,LIV,DEF,10.0,0.0,0.00,0.00,0.00,6.20,0.00,0.00,5.33,0.00,0.00,3.35,0.00,6.86,0.00,0.0,5.10,0.00,0.0,3.89,0.0,1.67,0.00,0.00,6.47,3.59,0.00,1.22,5.15,0.00,7.48,4.76,6.66,5.59,6.89,80.21
11,373,Ederson,MCI,GK,7.8,0.0,4.39,0.00,0.00,0.00,0.00,0.00,6.54,0.00,0.00,0.00,5.14,6.34,0.00,0.0,4.80,0.00,0.0,3.28,0.0,5.22,0.00,0.00,5.33,4.21,1.35,1.37,4.66,0.00,4.91,5.74,6.65,4.53,4.31,78.77
36,72,Pascal Groß,BHA,MID,8.2,0.0,0.00,0.00,0.00,0.00,7.02,6.20,0.00,0.00,6.09,0.00,0.00,4.62,0.00,0.0,0.00,6.55,0.0,0.00,0.0,6.63,0.00,0.00,5.61,6.50,0.00,1.23,3.69,5.18,0.00,4.33,5.69,5.33,3.07,77.74
29,100,Lewis Dunk,BHA,DEF,8.3,0.0,0.00,0.00,0.00,0.00,6.97,6.44,0.00,0.00,6.18,0.00,0.00,4.02,0.00,0.0,0.00,6.80,0.0,0.00,0.0,6.33,0.00,0.00,5.55,6.73,0.00,1.10,2.92,5.10,0.00,4.27,6.19,5.11,2.85,76.56
19,710,Rúben Dias,MCI,DEF,9.7,0.0,4.53,0.00,0.00,0.00,0.00,0.00,6.83,0.00,0.00,0.00,5.39,6.45,0.00,0.0,4.88,0.00,0.0,3.40,0.0,5.29,0.00,0.00,5.44,3.04,1.38,1.27,4.13,0.00,4.16,5.71,6.50,3.93,4.01,76.34
23,347,Alisson,LIV,GK,8.0,0.0,0.00,0.00,0.00,5.28,0.00,0.00,4.79,0.00,0.00,4.16,0.00,5.85,0.00,0.0,4.99,0.00,0.0,3.20,0.0,1.39,0.00,0.00,5.83,4.40,0.00,1.17,5.25,0.00,6.52,5.02,6.04,5.28,6.48,75.65


In [7]:
# To generate optimal team and plan:
# List all the sky IDs of all players to include in the initial squad (or leave empty), see prior_player_data.csv for ID reference
team = []

# Change how much you penalize the solver for making a transfer, alternatively set a hard limit by changing the total transfer allowance: ta_tot
transfer_cost = 8

# Example force transfer in constraints, either let the model decide when or specify a matchday
# fti = [{"sky_id": 610}, {"sky_id": 1072, "md": 3}, {"sky_id": 398, "md": 6}]
fti = None
fto = None

# To use FPL Kid data, download the 'SKY Fantasy EV - CSV' google sheet as fplkid.csv, add to data folder, and uncomment the following line. https://ko-fi.com/fplkid
# skymodel_output, md_map = read_fpl_kid_model(filepath='../data/fplkid.csv')
# Can cut off players with low EV to save solve time
skymodel_output, md_map = read_skymodel_output(max_ev_cutoff=0.3, max_ev_per_price_cutoff=0.3, initial_squad=team)
# Generate optimal plan
r3 = solve_sky_mp(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md='2024-04-23', flatten_mds_after='2024-03-02',
                  ta_tot=25, ta_gw=5, objective='decay', decay_base=0.95, transfer_cost=transfer_cost, vicecap_wt = 0.05,
                  exclusions=None, keeps=None, ban_teams=None, force_transfer_in=fti, force_transfer_out=fto, no_transfer_mds=None,
                  apply_noise=False, show_itb=True, show_non_team_ev=False)
display(r3['plan'])
print(f"Total xP: {r3['total_xp']}\t Eval: {r3['eval_score']}")
print(f"Total transfers made: {r3['transfers_made']}\t Transfer cost: {transfer_cost}")

NOTE: Initialized model multi_period.


sky_gw    23                   24  \
                                         fpl_gw    21                   21   
      ID Team Pos Value                    Name     1      2      3      4   
0    373  MCI   G   7.8                 Ederson         4.39      -      -   
1    741  BUR   G   6.6          James Trafford  5.9c                        
2     13  ARS   D   9.0          William Saliba     -      -      -  6.67c   
3    337  LIV   D  10.0  Trent Alexander-Arnold     -      -      -      -   
4    710  MCI   D   9.7              Rúben Dias     -  4.53v      -      -   
5    369  LIV   D   9.9         Virgil van Dijk     -      -      -      -   
6    100  BHA   D   8.3              Lewis Dunk     -      -      -      -   
7    398  MCI   M   9.4                   Rodri     -  5.54c      -      -   
8     72  BHA   M   8.2             Pascal Groß     -      -      -      -   
9     46  AVL   M   8.0            Douglas Luiz     -      -  5.52c      -   
10   182  CRY   M   8.2            Eberechi Eze                              
11  1280  MCI   F  12.0          Erling Haaland                              
12  1247  BOU   F   7.8         Dominic Solanke     -      -      -      -   
13   884  BRE   F   8.7              Ivan Toney     -      -      -  5.39v   
14                                        £ITBm   4.4    3.2    3.2    3.2   

                     25                   26                   27         \
                     22                   23                   24          
        5      6      7      8      9     10     11     12     13     14   
0       -      -      -   6.54      -      -      -   5.14   6.34      -   
1                                                                          
2       -      -   5.83                                                    
3    6.2c      -      -   5.33      -      -   3.35      -  6.86v      -   
4       -      -      -  6.83v      -      -      -   5.39   6.45      -   
5   6.06v      -      -   5.24      -      -  3.55v      -   6.69      -   
6       -  6.97v  6.44c      -      -  6.18c      -      -   4.02      -   
7       -      -      -   6.67      -      -      -  5.93v   6.68      -   
8       -  7.02c   6.2v      -      -  6.09v      -      -   4.62      -   
9       -      -   4.42      -      -   5.25      -      -      -  4.84c   
10                                                                         
11                       9.31c      -      -      -  7.16c  8.43c      -   
12   5.26      -      -      -  5.36c      -  6.59c      -   4.93      -   
13      -      -      -   4.69      -      -      -   3.86   4.58      -   
14    3.2    3.2    3.2    0.2    0.2    0.2    0.2    0.2    0.2    0.2   

             28                        29                          30  
             25                        26                          27  
      15     16     17     18     19   20     21     22     23     24  
0      -    4.8      -      -   3.28    -   5.22      -      -  32.98  
1                                                                      
2                                                                      
3      -    5.1      -      -  3.89v    -   1.67      -      -  36.01  
4      -   4.88      -      -    3.4    -   5.29      -      -  30.26  
5      -   5.27      -      -   3.76    -   1.61      -      -  36.39  
6      -      -   6.8c      -      -    -   6.33      -      -  31.23  
7      -  5.89v      -      -   3.68    -   6.05      -      -  35.69  
8      -      -  6.55v      -      -    -  6.63v      -      -  31.85  
9                                                                      
10  5.0c      -      -  4.69c      -    -   5.16      -      -  28.61  
11     -  7.32c      -      -  4.77c    -  7.36c      -      -   46.0  
12     -   4.15      -      -   3.29    -   4.01      -      -  37.32  
13     -   4.51      -      -   1.99    -      -  3.21c  1.38c  30.54  
14   0.0    0.0    0.0    0.0    0.0  0.0  

Total xP: 920.75	 Eval: 694.43
Total transfers made: 3	 Transfer cost: 8


In [ ]:
# To run multiple solves with noise:
team = []

transfer_cost = 9
# Choose a number of simulations to run (20 recommended at the minimum for more reliable results)
nsims = 50
# Choose the relative magnitude of applied noise, the higher the number the more diverse the generated plans will be (1 to 3 is common)
magnitude = 3

skymodel_output, md_map = read_skymodel_output(max_ev_cutoff=0.3, max_ev_per_price_cutoff=0.3, initial_squad=team)
# Generate sensitivity analysis
r4 = solve_sky_mp_noise(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md='2024-04-03', flatten_mds_after='2024-03-02',
                        ta_tot=25, ta_gw=5, objective='decay', decay_base=0.97, transfer_cost=transfer_cost, vicecap_wt = 0.05,
                        exclusions=None, keeps=None, ban_teams=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                        show_itb=True, show_non_team_ev=False,
                        seed_val=None, nsims=nsims, magnitude=magnitude)
results_dict = r4['results_dict']
display(r4['sensitivity_df'])
# display(r4['sensitivity_df_unformatted'])
print(f"Number of sims: {nsims}\t\tTransfer cost: {transfer_cost}\tNoise magnitude: {magnitude}\nAvg transfers made: {r4['avg_trf']}\t  xPts: {r4['avg_xp']}\t\tEval: {r4['avg_eval']}")

# Other Commands

In [ ]:
# To just see the upcoming fixtures:
r0 = generate_ticker()
r0['formatted_fixtures']

In [ ]:
# To see a breakdown of EV for a given player and fixture:
prior_player_data = pd.read_csv('../data/prior_player_data.csv')
team_data = pd.read_csv('../data/team_priors.csv')
# Find a players sky_id in prior_player_data.csv
# opp_team is case sensitive, uppercase and lowercase implying home and away respectively
# r1 = sky_xP_calc(sky_id=377, opp_team='BHA', prior_player_data=prior_player_data, team_data=team_data, xMins=85, xP_breakdown=True)
r1 = sky_xP_calc(sky_id=412, opp_team='SHU', prior_player_data=prior_player_data, team_data=team_data, xMins=85, xP_breakdown=True)


r1['xP_breakdown']

In [ ]:
# To view the upcoming captaincy matrix first generate player EV then:
generate_cap_matrix(last_md=13, max_ev_diff=1)